# SI 618: Data Manipulation and Analysis
## 06 - Categorical Data & Text Processing 
### Contingency tables, crosstabs, mosaic plots and chi-squared
### Introduction to text processing
### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a> This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.
    
Version 2021.03.01.1.CT


# Categorical Data

Categorical data are those that can take on one of a limited number of values (i.e. categories) (Wikipedia)

Examples: blood type (A, B, AB, O); types of rock (sedimentary, metamorphic, igneous).

## Contingency tables, crosstabs, and chi-square

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

Let's generate a data frame to play with:

In [ ]:
df = pd.DataFrame({'color' : ['red', 'green', 'green', 'black'] * 6,
                   'make' : ['ford', 'toyota', 'dodge'] * 8,
                   'vehicleClass' : ['suv', 'suv', 'suv', 'car', 'car', 'truck'] * 4})

In [ ]:
df

One of the most basic transformations we can do is a crosstab:

In [ ]:
ct = pd.crosstab(df.color,df.vehicleClass)
ct

Notice how similar it is to pivoting.  In fact, go ahead and use ```pivot_table``` to do the same sort of transformation:

### <font color="magenta">Q1: Use ```pivot_table``` to create a DataFrame similar to the one from the ```crosstab``` above:

In [1]:
# insert your code here

As usual, we would like to visualize our results:

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(ct,annot=True)

You might want to investigate other palettes, see https://seaborn.pydata.org/tutorial/color_palettes.html for more details.



In [ ]:
sns.heatmap(ct,annot=True,cmap=sns.cubehelix_palette())

### Titanic data

One of the more popular datasets that we use for experimenting with crosstabs is the 
survivor data from the Titanic disaster:

In [ ]:
titanic = pd.read_csv('https://raw.githubusercontent.com/umsi-data-science/data/main/titanic.csv')

Let's create a crosstab of the data:

In [ ]:
ct = pd.crosstab(titanic.passtype,titanic.status,margins=False)
ct

In [ ]:
sns.heatmap(ct,annot=True,cmap=sns.cubehelix_palette())

Does scientific notation bother you?  Change the format with the ```fmt=``` argument:

In [ ]:
sns.heatmap(ct,annot=True,cmap=sns.cubehelix_palette(),fmt='d')

### <font color="magenta"> Q2: Is there something interesting here?</font>

Insert your answer here.

In addition to the heatmap shown above, we can use a mosaic plot to visualize 
contingency tables:

In [ ]:
from statsmodels.graphics.mosaicplot import mosaic
t = mosaic(titanic, ['passtype','status'],title='titanic survival')

In [ ]:
# slightly easier to read
props = lambda key: {'color': 'r' if 'alive' in key else 'gray'}
t = mosaic(titanic, ['passtype','status'],title='titanic survival',properties=props)

Let's take a look at the "expected" values for each cell.  That is,
the values that we would expect if there was no effect of "passtype" on "status".  To do this, let's take a look at the marginal totals:

In [ ]:
ct = pd.crosstab(titanic.passtype,titanic.status,margins=False)
ct

The **expected** value for each cell (i.e. the value that you would expect if there was no interaction between passtype and status in this dataset) can be calculated by taking the row total multiplied by the column total and divided by the overall total.  

So we would get the following expected value for alive crew:

In [ ]:
exp = ct['All'].loc['crew'] * ct['alive'].loc['All'] / ct['All'].loc['All']

In [ ]:
exp

You could repeat this for each cell (or write code to do so), but you get the idea.

## Let's talk about $\chi^2$

Finally, we can go beyond visual exploration and apply analytic tests to see if the 
observed values differ from the expected ones.  The chi-square test sums the squares of the differences
between the observed and expected values, normalized for the expected values.

Our null hypothesis is that there is no difference in survivorship based on passage type.

Here's a video resource that explains chi-squared:

In [ ]:
from IPython.display import YouTubeVideo
vid = YouTubeVideo("VskmMgXmkMQ")
display(vid)

In [ ]:
from scipy.stats import chi2_contingency
chi2, p, dof, ex = chi2_contingency(ct)
print("chi2 = ", chi2)
print("p-val = ", p)
print("degree of freedom = ",dof)

As a bonus, we also get a DataFrame of the expected values:

In [ ]:
pd.DataFrame(ex)

## Let's apply these ideas to another dataset

For this component, we'll use the Comic Characters data set:

In [ ]:
comic_characters = pd.read_csv("https://raw.githubusercontent.com/umsi-data-science/data/main/comic_characters.csv", index_col="id")
comic_characters.head(1)

### Example

We'd like to know which publisher uses different 'identity' types for their characters? Have DC characters appeared more publicly? What is the average number of times Marvel Characters appeared known to authorities identity?

In [ ]:
comic_characters.groupby(['Identity','publisher'])['appearances'].mean().unstack().fillna(0)

Alternatively, we can use .pivot_table(). For example:

In [ ]:
avg_appearance_per_identity = comic_characters.pivot_table(index='Identity', 
                                                          columns='publisher', 
                                                          values='appearances',
                                                          aggfunc='mean')
avg_appearance_per_identity.fillna(0).head()

For .pivot_table(), you need to specify these four arguments:
1. index: the field that will become the index of the output table
2. columns: the field that will become the columns of the output table
3. values: the field to be aggregated/summarized
4. aggfunc: the aggregation operator applied to values, if there are more than 1 entry corresponding to each (index, column) pair, such as "mean", "count", "max"
    

### Let's warm up with a few groupby and pivot_table exercises:

What is the total number of appearances of characters for each publisher?

In [67]:
a = comic_characters.groupby('publisher')[['appearances']].sum()
a.head()

,appearances
publisher,
DC,154532.0
Marvel,260270.0


What is the total number of appearances of characters by each publisher in each year? Output a table.

In [68]:
b = comic_characters.groupby(['year','publisher']).sum().fillna(0)
b

appearances
year   publisher             
1935.0 DC               125.0
1936.0 DC               240.0
1937.0 DC               458.0
1938.0 DC              1639.0
1939.0 DC              4067.0
...                       ...
2011.0 Marvel          1890.0
2012.0 DC                17.0
       Marvel          1072.0
2013.0 DC                 1.0
       Marvel          1177.0

[154 rows x 1 columns]

### Can we improve that output?

Try unstacking it...

In [73]:
b = comic_characters.groupby(['year','publisher']).sum().unstack().fillna(0)
b.head(10)

appearances        
publisher          DC  Marvel
year                         
1935.0          125.0     0.0
1936.0          240.0     0.0
1937.0          458.0     0.0
1938.0         1639.0     0.0
1939.0         4067.0  1013.0
1940.0         9123.0  2160.0
1941.0         6945.0  5371.0
1942.0         4068.0   401.0
1943.0         1475.0   541.0
1944.0          274.0   960.0

Construct a contingency table of sex and character alignment normalized by all values.

Display the normalized values in percentage (%) format. Use brief sentences to explain your findings.  

Hint: use "normalize=all" in your crosstab statement.  What does normalize do? (read the docs)

In [74]:
ct = pd.crosstab(comic_characters['sex'], comic_characters['align'], normalize='all')
ct.apply(lambda x: x.apply(lambda y:  str('{:,.2f}'.format(y*100)) + '%' ))

align,Bad Characters,Good Characters,Neutral Characters,Reformed Criminals
sex,,,,
Agender Characters,0.10%,0.05%,0.07%,0.00%
Female Characters,8.22%,13.01%,4.37%,0.01%
Genderfluid Characters,0.00%,0.01%,0.01%,0.00%
Genderless Characters,0.06%,0.03%,0.02%,0.00%
Male Characters,39.51%,25.13%,9.40%,0.01%
Transgender Characters,0.01%,0.00%,0.00%,0.00%


### <font color="magenta">Q3: Create a mosaic plot of align and alive.  Conduct a $\chi^2$ test of ```align``` and ```alive```. Please specify your (null and alternative) hypotheses and explain your findings.

In [ ]:
# insert your code here

Insert your interpretation here.

# BREAK!

# Text Processing I: Basics and Regular Expressions

As usual, let's load up some data:

In [ ]:
import pandas as pd

In [ ]:
reviews = pd.read_csv('https://raw.githubusercontent.com/umsi-data-science/data/main/amazon_food_reviews.zip')

Let's take a really small sample, just so we can experiment with the various 

In [ ]:
reviews_sample = reviews.head(10)

In [ ]:
reviews_sample

Let's review some basic string functionality from Pandas that can be applied to any Series or Index:

In [ ]:
reviews_sample.ProfileName.str.lower()

In [ ]:
reviews_sample.ProfileName.str.upper()

In [ ]:
reviews_sample.Summary.str.len()

Remember, the ```columns``` attribute of a DataFrame is an Index object, which means that we can use str operators on the column names:

In [ ]:
reviews_sample.columns

In [ ]:
reviews_sample.columns.str.lower()

Notice that the "User Id" column of the dataframe looks weird:  it has a space in the middle *and* at the end.  Columns that are named like that will invariable trip us up in downstream (i.e. later) analyses, so it's wise to correct them now.  Something like the following can help:

In [ ]:
reviews_sample.columns.str.strip().str.lower().str.replace(' ','_')

And we can assign that back to the columns attribute to actually rename the columns:


In [ ]:
reviews_sample.columns = reviews_sample.columns.str.strip().str.lower().str.replace(' ','_')

In [ ]:
reviews_sample

### Splitting and Replacing Strings

Sometimes, we want to split strings into lists.  We might want to do that with the "summary" column:

In [ ]:
reviews_sample.productid.str.split('00')

In [ ]:
reviews_sample.productid.str.split('00').str.get(1)

Equivalently:

In [ ]:
reviews_sample.productid.str.split('00').str[1]

### Replace (regex time!)

In [ ]:
reviews_sample.summary.str.lower().str.replace('dog','health')

In [ ]:
reviews_sample.summary.str.lower().str.replace('dog|taffy','health')

### Extracting Substrings

In [ ]:
reviews_sample.summary.str.extract(r'(Dog)')

In [ ]:
reviews_sample.summary.str.extract(r'(Dog|Taffy)')

In [ ]:
reviews_sample.summary.str.extract(r'(Dog|[Tt]affy)')

In [ ]:
# returns a Series
reviews_sample.summary.str.extract(r'(Dog|[Tt]affy)', expand = False)

In [ ]:
reviews_sample.summary.str.extractall(r'(Dog|[Tt]affy)')

In [ ]:
reviews_sample.summary.str.extractall(r'(as)')

### Testing for Strings that Match or Contain a Pattern

In [ ]:
reviews_sample.text

In [ ]:
pattern = r'[Gg]ood'

In [ ]:
reviews_sample.text.str.contains(pattern)

In [ ]:
reviews_sample.text.str.match(pattern)

In [ ]:
pattern = r'.*[Gg]ood.*'

In [ ]:
reviews_sample.text.str.match(pattern)

#### Helpful resources:
- Pandas text documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html
- Regex Cheat Sheet: https://regexr.com/

### <font color="magenta">Q4: How many rows from the Amazon Food Reviews data set contain the word ```vegan``` in the  ```Text``` column?</font>

In [ ]:
# insert your code here

### <font color="magenta">Q5: How many rows from the Amazon Food Reviews data set contain HTML tags in the ```Text``` column?</font>
Hint: here's how to find an HTML tag: https://stackoverflow.com/questions/45999415/removing-html-tags-in-pandas.  Explain what the regex means.

In [2]:
# insert your code here

### <font color="magenta">  Stretch: Remove all HTML tags from the Amazon Food Reviews text column and save the results to a column called text_no_html.

In [3]:
# insert your code here